In [1]:
%load_ext autoreload
%autoreload 2

import helper_final1 as helper
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Cropping2D, Conv2D, ZeroPadding2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Flatten, Dropout, Dense
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import keras.backend as k
from keras.initializers import TruncatedNormal
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers import Activation
from keras.layers.advanced_activations import LeakyReLU
import h5py
import pdb
from sklearn.utils import class_weight

#turn off warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline


Using TensorFlow backend.


# Read in data, format, & visualize

In [2]:
emotion_dict = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'sad', 5:'surprise', 6:'neutral'}
image_shape = (48,48,1)
n_classes = len(emotion_dict)
n_features = np.product(image_shape)

# read in data
file_path = './data/Kaggle-FER/fer2013.csv'
data_raw = helper.read_in_data(file_path) # data is a tuple of (x_list, y_list, usage_list)

# remove blanks, over sample, convert data to numpy array
data_formatted = helper.format(data_raw, image_shape, n_classes) # return tuple of (x_array, y_array, usage_list)

# number of example for each class
label, count = np.unique(data_formatted[1], return_counts=True)
dict(zip(label, count))

# split into train & test sets. Test set = public, private set is used for final testing, to be consistent with the competition
(x_train, y_train), (x_test, y_test), (x_private, y_private) = helper.create_train_test(data_formatted, n_classes)

# one-hot encode
y_train_onehot = keras.utils.to_categorical(y_train, n_classes)
y_test_onehot = keras.utils.to_categorical(y_test, n_classes)
y_private_onehot = keras.utils.to_categorical(y_private, n_classes)

# Build model & train

# 1st place model

## l2=0.001

In [3]:
########## arguments ##########
seed = 2
kernel_initializer = TruncatedNormal(seed=seed)
kernel_regularizer = regularizers.l2(0.001)
batch_norm = True

cp_name = './models/l20.001.hdf5'
model_name = 'l20.001'

########## build model ##########
model = helper.build_model(model_name, kernel_initializer=kernel_initializer
                           , kernel_regularizer=kernel_regularizer, batch_norm=batch_norm
                           , seed=seed)
model.summary
########## preprocess data, compile & train model ##########
# history = helper.train(x_train, y_train_onehot, x_test, y_test_onehot, x_private, y_private_onehot
#                        , model, cp_name)

########## visualize metrics ##########
# helper.visualize_metrics(history)

<bound method Container.summary of <keras.models.Sequential object at 0x0000022BBD930FD0>>

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_crop (Cropping2D)     (None, 42, 42, 1)         0         
_________________________________________________________________
block1_conv (Conv2D)         (None, 42, 42, 32)        832       
_________________________________________________________________
block1_bn (BatchNormalizatio (None, 42, 42, 32)        128       
_________________________________________________________________
block1_relu (LeakyReLU)      (None, 42, 42, 32)        0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 21, 21, 32)        0         
_________________________________________________________________
block2_pad (ZeroPadding2D)   (None, 23, 23, 32)        0         
_________________________________________________________________
block2_conv (Conv2D)         (None, 20, 20, 32)        16416     
__________

In [5]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.